In [7]:
import pandas as pd

import json

from contoso_chat.chat_request import get_response

from evaluators.custom_evals.coherence import coherence_evaluation
from evaluators.custom_evals.relevance import relevance_evaluation
from evaluators.custom_evals.fluency import fluency_evaluation
from evaluators.custom_evals.groundedness import groundedness_evaluation
from evaluators.custom_evals.politeness import politeness_evaluation

import jsonlines

## Get output from data and save to results jsonl file

In [8]:

def load_data():
    data_path = "./evaluators/data_small.jsonl"

    df = pd.read_json(data_path, lines=True)
    df.head()
    return df

In [9]:

def create_response_data(df):
    results = []

    for index, row in df.iterrows():
        customerId = row['customerId']
        question = row['question']
        
        # Run contoso-chat/chat_request flow to get response
        response = get_response(customerId=customerId, question=question, chat_history=[])
        print(response)
        
        # Add results to list
        result = {
            'question': question,
            'context': response["context"],
            'answer': response["answer"]
        }
        results.append(result)

    # Save results to a JSONL file
    with open('result_small.jsonl', 'w') as file:
        for result in results:
            file.write(json.dumps(result) + '\n')
    return results

In [10]:
def evaluate():
    # Evaluate results from results file
    results_path = 'result_small.jsonl'
    results = []
    with open(results_path, 'r') as file:
        for line in file:
            print(line)
            results.append(json.loads(line))

    for result in results:
        question = result['question']
        context = result['context']
        answer = result['answer']
        
        groundedness_score = groundedness_evaluation(question=question, answer=answer, context=context)
        fluency_score = fluency_evaluation(question=question, answer=answer, context=context)
        coherence_score = coherence_evaluation(question=question, answer=answer, context=context)
        relevance_score = relevance_evaluation(question=question, answer=answer, context=context)
        politeness_score = politeness_evaluation(question=question, answer=answer, context=context)
        
        result['groundedness'] = groundedness_score
        result['fluency'] = fluency_score
        result['coherence'] = coherence_score
        result['relevance'] = relevance_score
        result['politeness'] = politeness_score

    # Save results to a JSONL file
    with open('result_evaluated_small.jsonl', 'w') as file:
        for result in results:
            file.write(json.dumps(result) + '\n')

    with jsonlines.open('eval_results_small.jsonl', 'w') as writer:
        writer.write(results)
    # Print results

    df = pd.read_json('result_evaluated_small.jsonl', lines=True)
    df.head()
    
    return df

In [11]:
def create_summary(df):
    print("Evaluation summary:\n")
    print(df)
    # drop question, context and answer
    mean_df = df.drop(["question", "context", "answer"], axis=1).mean()
    print("\nAverage scores:")
    print(mean_df)
    df.to_markdown('eval_results_small.md')
    with open('eval_results_small.md', 'a') as file:
        file.write("\n\nAverages scores:\n\n")
    mean_df.to_markdown('eval_results_small.md', 'a')

    print("Results saved to result_evaluated.jsonl")

In [12]:
# create main funciton for python script
if __name__ == "__main__":

   test_data_df = load_data()
   response_results = create_response_data(test_data_df)
   result_evaluated = evaluate()
   create_summary(result_evaluated)


Starting get_response
signature:
"contoso_chat.chat_request.get_response"
inputs:
{
    "customerId": 4,
    "question": "tell me about your hiking jackets",
    "chat_history": []
}
getting customer...
Starting get_customer
signature:
"contoso_chat.chat_request.get_customer"
inputs:
{
    "customerId": 4
}
result:
{
    "id": "4",
    "firstName": "Sarah",
    "lastName": "Lee",
    "age": 38,
    "email": "sarahlee@example.com",
    "phone": "555-867-5309",
    "address": "321 Maple St, Bigtown USA, 90123",
    "membership": "Platinum",
    "orders": [
        {
            "id": 26,
            "productId": 7,
            "quantity": 1,
            "total": 100,
            "date": "2/5/2023",
            "name": "CozyNights Sleeping Bag",
            "unitprice": 100,
            "category": "Sleeping Bags",
            "brand": "CozyNights",
            "description": "Embrace the great outdoors in any season with the lightweight CozyNights Sleeping Bag! This durable three-season 